In [1]:
import cv2
import numpy as np

def estimate_homography(image1_path, image2_path, ratio_threshold=0.75, ransac_threshold=5.0):
  
    # Load the images
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    
    # Detect and compute keypoints and descriptors using ORB
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)
    
    # Match descriptors using BFMatcher with L2 norm
    bf = cv2.BFMatcher(cv2.NORM_L2)
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # Apply the ratio test to filter matches
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)
    
    if len(good_matches) < 4:
        print("Not enough matches to estimate homography.")
        return None, None, good_matches, keypoints1, keypoints2, img1, img2

    # Extract matched points
    src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Estimate the homography matrix using RANSAC
    homography_matrix, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, ransac_threshold)
    
    return homography_matrix, mask, good_matches, keypoints1, keypoints2, img1, img2

def draw_matches_with_inliers(matches, mask, keypoints1, keypoints2, img1, img2):
    """
    Draws matches between two images, highlighting inliers and excluding outliers.

    Parameters:
    - matches: List of matches.
    - mask: Inlier mask from RANSAC.
    - keypoints1: Keypoints from the first image.
    - keypoints2: Keypoints from the second image.
    - img1: The first image (grayscale).
    - img2: The second image (grayscale).
    """
    # Keep only inliers
    inlier_matches = [match for match, m in zip(matches, mask.ravel()) if m == 1]
    match_img = cv2.drawMatches(img1, keypoints1, img2, keypoints2, inlier_matches, None,
                                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return match_img

if __name__ == "__main__":
    # Paths to the input images
    image1_path = "/home/student/Pictures/Screenshots/Screenshot from 2024-12-04 11-56-45.png"  # Replace with your image path
    image2_path = "/home/student/Pictures/Screenshots/Screenshot from 2024-12-05 09-48-55.png"  # Replace with your image path

    # Estimate the homography matrix
    homography_matrix, mask, good_matches, keypoints1, keypoints2, img1, img2 = estimate_homography(
        image1_path, image2_path
    )
    
    if homography_matrix is not None:
        print("Estimated Homography Matrix:")
        print(homography_matrix)
        print(f"Number of inliers: {np.sum(mask)} / {len(good_matches)}")
        
        # Draw matches with inliers
        match_img = draw_matches_with_inliers(good_matches, mask, keypoints1, keypoints2, img1, img2)
        cv2.imshow("Inlier Matches", match_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Failed to estimate homography.")


Estimated Homography Matrix:
[[-3.43896053e+00 -6.46812038e-02  9.06831689e+01]
 [-1.75718828e+00 -5.76913383e+00  3.85474087e+02]
 [-1.27793088e-02 -1.26120870e-02  1.00000000e+00]]
Number of inliers: 4 / 7


QSocketNotifier: Can only be used with threads started with QThread
